In [ ]:
import requests
import random
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
import random
import csv
from datetime import datetime
import os.path
import time
import pandas as pd
from random import randrange

In [ ]:
def load_user_agents(filepath):
  # Load user agents list from a txt file
  file = open(filepath, "r")
  agents = file.read()
  user_agent_list = agents.split("\n")
  file.close()
  return user_agent_list

In [ ]:
user_agent_list = load_user_agents("user-agents_chrome_browser_100-0.txt")

In [ ]:
# Get free proxies

res = requests.get('https://free-proxy-list.net')
content = bs(res.text, 'lxml')
table = content.find('table')
rows = table.find_all('tr')
cols = [[col.text for col in row.find_all('td')] for row in rows]

proxieslist = []

for col in cols:
  try:
    if col[4] == 'elite proxy' and col[6] == 'yes':
      proxieslist.append('https://' + col[0] + ':' + col[1])
  except:
    pass

In [ ]:
proxieslist

### Run every 5 minutes to be safe

In [ ]:
attraction = 'adventure_cove'
file_path = 'adventurecovetest.csv'

# Open csv file to write in
with open(file_path, 'a', encoding="utf-8", newline='') as f:
  csv_writer = csv.writer(f)
  csv_writer.writerow(['date', 'user', 'review', 'attraction', 'source'])

page = 0
count_reviews = 0

while True:
  try:
    page += 1
    url = f'https://www.klook.com/v1/usrcsrv/activities/120/reviews?page={page}&limit=8&star_num=&lang=id_ID&sort_type=1&only_image=false'

    for _ in user_agent_list:
      #Pick a random user agent
      user_agent = random.choice(user_agent_list)
      #Set the headers 
      headers = {'User-Agent': user_agent, 'referer': "https://www.google.com"}

    time.sleep(randrange(10,30))
    source = requests.get(url, headers=headers)
    
    data = source.json()

    content = data['result']['item']

    review_count = len(content)
    
    for i in range(review_count):
      date = content[i]['date']
      author = content[i]['author']
      review = content[i]['content']
      print(date, author, review)
      
      # Open csv file to write in
      with open(file_path, 'a', encoding="utf-8", newline='') as f:
        csv_writer = csv.writer(f)
        csv_writer.writerow([date, author, review, attraction, 'klook']) 
  except:
    break

# Scrape in Function

In [ ]:
def scrape_klook(attraction, id):
  file_path = f'{attraction.lower().replace(" ", "_")}.csv'

  # Open csv file to write headers
  with open(f'data/raw/{file_path}', 'a', encoding="utf-8", newline='') as f:
    csv_writer = csv.writer(f)
    csv_writer.writerow(['date', 'user', 'review', 'attraction', 'source'])

  page = 0
  # count_reviews = 0

  while True:
    try:
      page += 1
      url = 'https://www.klook.com/v1/usrcsrv/activities/' + str(id) + '/reviews?page=' + str(page) + '&limit=8&star_num=&lang=id_ID&sort_type=1&only_image=false'

      for _ in user_agent_list:
        #Pick a random user agent
        user_agent = random.choice(user_agent_list)
        #Set the headers 
        headers = {'User-Agent': user_agent, 'referer': "https://www.google.com"}

      time.sleep(randrange(60,120))
      source = requests.get(url, headers=headers)
      
      data = source.json()
      print(data)

      content = data['result']['item']

      review_count = len(content)
      
      for i in range(review_count):
        date = content[i]['date']
        author = content[i]['author']
        review = content[i]['content']
        print(date, author, review)
        
        # Open csv file to write in
        with open(f'data/raw/{file_path}', 'a', encoding="utf-8", newline='') as f:
          csv_writer = csv.writer(f)
          csv_writer.writerow([date, author, review, attraction, 'klook']) 
    except:
      break

In [ ]:
df = pd.read_csv('attractions_id.csv', header=None, names=["Attraction", "ID"])
df

In [10]:
for index, row in df.iterrows():
  time.sleep(randrange(300,420))
  scrape_klook(row['Attraction'], row['ID'])

In [ ]:
scrape_klook('test', 139)